<a href="https://colab.research.google.com/github/awais-yaqoob-ml/ml-codes/blob/main/mobilnetv2tfdogcat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.0
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
import tensorflow_datasets as tfds

(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteS151L5/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [3]:
IMG_SIZE = 160
def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE,IMG_SIZE))
  return image, label

In [4]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

In [13]:
BATCH_SIZE = 64

train_batches = train.shuffle(1000).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)


In [6]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(160,160,3),
    include_top=False,
    weights="imagenet"
)

9406464/9406464 [==============================] - 0s 0us/step


In [7]:
base_model.trainable = False
base_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 80, 80, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 80, 80, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 80, 80, 32)   0           ['bn_Conv1[0][

In [8]:
model = tf.keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(1)
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_160 (Funct  (None, 5, 5, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________


In [9]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [ ]:
history = model.fit(
    train_batches,
    epochs=20,
    validation_data=validation_batches
)

Epoch 1/20
291/291 [==============================] - 38s 123ms/step - loss: 0.0306 - accuracy: 0.9895 - val_loss: 0.0434 - val_accuracy: 0.9862
Epoch 2/20
291/291 [==============================] - 38s 126ms/step - loss: 0.0289 - accuracy: 0.9897 - val_loss: 0.0422 - val_accuracy: 0.9867
Epoch 3/20
291/291 [==============================] - 46s 152ms/step - loss: 0.0279 - accuracy: 0.9900 - val_loss: 0.0419 - val_accuracy: 0.9867
Epoch 4/20
291/291 [==============================] - 48s 160ms/step - loss: 0.0272 - accuracy: 0.9903 - val_loss: 0.0426 - val_accuracy: 0.9858
Epoch 5/20
291/291 [==============================] - 37s 124ms/step - loss: 0.0261 - accuracy: 0.9905 - val_loss: 0.0432 - val_accuracy: 0.9854
Epoch 6/20
291/291 [==============================] - 37s 122ms/step - loss: 0.0253 - accuracy: 0.9911 - val_loss: 0.0443 - val_accuracy: 0.9858
Epoch 7/20
291/291 [==============================] - 38s 127ms/step - loss: 0.0243 - accuracy: 0.9912 - val_loss: 0.0458 - val_ac

In [ ]:
model.save('dogvscats.h5')
nmodel = keras.models.load_model('dogvscats.h5')
#predict function here